In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import cv2
import numpy as np
import os
from scipy.spatial import distance

# Load the CSV file
data = pd.read_csv('images.csv')

# Preprocess the images
image_directory = 'D:\Myntra'  # Update this with the actual path to your images

# Create image data generator
datagen = ImageDataGenerator(rescale=0.2)



In [2]:
import yt_dlp

def download_video(url, save_path):
    ydl_opts = {
        'outtmpl': save_path + '/vid.%(ext)s',  # Save all videos as vid.mp4
        'format': 'best[ext=mp4][height<=1080]',  # Download the best progressive stream available in MP4 format
        'merge_output_format': 'mp4',  # Ensure output format is MP4 (should not need merging for progressive streams)
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
        print(f'Video downloaded successfully and saved to {save_path}')
    except Exception as e:
        print(f'An error occurred: {e}')

# Example usage
video_url = 'https://www.youtube.com/shorts/rHogsxCI6gA'  # Replace with your video URL
save_path = 'D:/Myntra'  # Replace with your desired save directory
download_video(video_url, save_path)


[youtube] Extracting URL: https://www.youtube.com/shorts/rHogsxCI6gA
[youtube] rHogsxCI6gA: Downloading webpage
[youtube] rHogsxCI6gA: Downloading ios player API JSON
[youtube] rHogsxCI6gA: Downloading m3u8 information
[info] rHogsxCI6gA: Downloading 1 format(s): 18
[download] D:\Myntra\vid.mp4 has already been downloaded
[download] 100% of  928.96KiB
Video downloaded successfully and saved to D:/Myntra


In [75]:
import shutil
def capture_screenshots(video_path, interval_sec=5, output_folder="screenshots"):
    print(f"Capturing screenshots from video: {video_path}")

    # Check if video file exists
    if not os.path.exists(video_path):
        print(f"Video file does not exist: {video_path}")
        return

    # Create output folder if it doesn't exist
    if os.path.exists(output_folder):
        # Delete old screenshots if any
        shutil.rmtree(output_folder)
    os.makedirs(output_folder)

    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Check if video file was opened successfully
    if not cap.isOpened():
        print("Error opening video file.")
        return

    # Get frame rate of the video
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    interval_frames = interval_sec * fps

    print(f"Video frame rate: {fps} frames per second")

    frame_count = 0
    screenshot_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % interval_frames == 0:
            screenshot_path = f"{output_folder}/screenshot_{screenshot_count}.jpg"
            cv2.imwrite(screenshot_path, frame)
            screenshot_count += 1
            print(f"Saved screenshot {screenshot_count} at frame {frame_count}")

        frame_count += 1

    cap.release()

In [8]:
# Preprocess images and labels
images = []
labels = []
for idx, row in data.iterrows():
    img_path = os.path.join(image_directory, row['image'] + '.jpg')  # Assuming images are in jpg format
    if os.path.exists(img_path):
        img = cv2.imread(img_path)
        img = cv2.resize(img, (128, 128))
        images.append(img)
        labels.append(row['label'])

images = np.array(images)
labels = pd.get_dummies(labels).values

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)



In [66]:
# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(labels[0]), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))

# Save the model
model.save('clothing_model3.h5')




Epoch 1/20


c:\Users\icyga\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


17/17 ━━━━━━━━━━━━━━━━━━━━ 4s 169ms/step - accuracy: 0.0783 - loss: 66.0700 - val_accuracy: 0.1692 - val_loss: 2.8582
Epoch 2/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 160ms/step - accuracy: 0.2381 - loss: 2.7066 - val_accuracy: 0.2538 - val_loss: 2.7325
Epoch 3/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 154ms/step - accuracy: 0.2653 - loss: 2.5461 - val_accuracy: 0.2385 - val_loss: 2.7421
Epoch 4/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 155ms/step - accuracy: 0.2957 - loss: 2.5015 - val_accuracy: 0.2846 - val_loss: 2.5227
Epoch 5/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 165ms/step - accuracy: 0.4081 - loss: 2.0250 - val_accuracy: 0.3154 - val_loss: 2.5279
Epoch 6/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 173ms/step - accuracy: 0.4730 - loss: 1.8049 - val_accuracy: 0.3308 - val_loss: 2.5192
Epoch 7/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 166ms/step - accuracy: 0.5311 - loss: 1.6566 - val_accuracy: 0.3462 - val_loss: 2.4128
Epoch 8/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 171ms/step - accuracy: 0.5580 - loss: 1.4074 - val_accuracy: 0.3462 - val

In [67]:
from sklearn.cluster import KMeans
from scipy.spatial import distance
from tensorflow.keras.models import load_model
import webcolors
from scipy.spatial import KDTree

In [68]:
# Load the saved model
loaded_model = load_model('clothing_model3.h5')

# Function to detect clothing type
def detect_clothing_type(image_path, model):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (128, 128))
    img = np.expand_dims(img, axis=0)
    prediction = model.predict(img)
    predicted_label = np.argmax(prediction, axis=1)
    label_map = {i: l for i, l in enumerate(data['label'].unique())}
    return label_map[predicted_label[0]]

# Predefined color dictionary
COLORS = {
    "black": (0, 0, 0), "white": (255, 255, 255), "red": (255, 0, 0), 
    "lime": (0, 255, 0), "blue": (0, 0, 255), "yellow": (255, 255, 0), 
    "cyan": (0, 255, 255), "magenta": (255, 0, 255), "silver": (192, 192, 192), 
    "gray": (128, 128, 128), "maroon": (128, 0, 0), "olive": (128, 128, 0), 
    "green": (0, 128, 0), "purple": (128, 0, 128), "teal": (0, 128, 128), 
    "navy": (0, 0, 128)
}


def closest_color(requested_color):
    min_colors = {}
    for name, rgb in COLORS.items():
        rd = (rgb[0] - requested_color[0]) ** 2
        gd = (rgb[1] - requested_color[1]) ** 2
        bd = (rgb[2] - requested_color[2]) ** 2
        min_colors[(rd + gd + bd)] = name
    return min_colors[min(min_colors.keys())]

def detect_color(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (128, 128))
    data = np.reshape(img, (-1, 3))
    data = np.float32(data)

    # Apply k-means clustering to find the dominant color
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)
    k = 1
    _, label, center = cv2.kmeans(data, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
    center = np.uint8(center)
    dominant_color = center[0]

    # Find the closest named color
    color_name = closest_color(dominant_color)
    return color_name




In [76]:
capture_screenshots('vid.mp4')

Capturing screenshots from video: vid.mp4
Video frame rate: 30 frames per second
Saved screenshot 1 at frame 0
Saved screenshot 2 at frame 150
Saved screenshot 3 at frame 300


In [77]:
def main():
    

    # Define class labels (from your training phase)
    class_indices = {'Blazer': 0, 'Blouse': 1, 'Body': 2, 'Dress': 3, 'Hat': 4,  'Longsleeve': 6, 'Not sure': 7, 'Other': 8, 'Outwear': 9, 'Pants': 10, 'Polo': 11, 'Shirt': 12, 'Shoes': 13, 'Shorts': 14, 'Skip': 15, 'Skirt': 16, 'T-Shirt': 17, 'Top': 18, 'Undershirt': 19}
    class_labels = {v: k for k, v in class_indices.items()}

    # Predict clothing types and colors for captured screenshots
    screenshot_folder = "screenshots"
    best_prediction = None
    highest_confidence = 0

    for screenshot in os.listdir(screenshot_folder):
        if screenshot.endswith('.jpg'):
            screenshot_path = os.path.join(screenshot_folder, screenshot)
            predicted_label= detect_clothing_type(screenshot_path, loaded_model)
            dominant_color = detect_color(screenshot_path)
            print(f"Predicted clothing type for {screenshot}: {predicted_label}, Dominant color: {dominant_color}")



if __name__ == "__main__":
    main()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Predicted clothing type for screenshot_0.jpg: Dress, Dominant color: silver
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Predicted clothing type for screenshot_1.jpg: Dress, Dominant color: gray
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Predicted clothing type for screenshot_2.jpg: Body, Dominant color: silver


In [70]:
# Test the function
image_path = '2058.jpg'
clothing_type = detect_clothing_type(image_path, loaded_model)
color = detect_color(image_path)

print(f"Clothing type: {clothing_type}")
print(f"Color: {color}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Clothing type: Pants
Color: silver
